# QbraidProvider

In [21]:
import qbraid

from qbraid.runtime.native import QbraidProvider

In [4]:
qbraid.__version__

'0.7.0.dev20240516020308'

In [22]:
provider = QbraidProvider()

provider.get_devices()

[<qbraid.runtime.native.device.QbraidDevice('qbraid_qir_simulator')>]

In [ ]:
aws_device = provider.get_device("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

type(aws_device)

In [ ]:
ibm_device = provider.get_device("ibm_brisbane")

type(ibm_device)

## BraketProvider

In [ ]:
from qbraid.providers.aws import BraketProvider, BraketDevice, get_quantum_task_cost

braket_provider = BraketProvider()

### Query AWS devices

In [ ]:
braket_provider.get_devices(statuses="ONLINE", types=["QPU"])

In [ ]:
braket_device = BraketDevice(aws_device)

braket_device.vendor, braket_device.provider, braket_device.name

In [ ]:
braket_device.queue_depth()

In [ ]:
_, _, next_availability_switch_utc = braket_device.availability_window()

print(next_availability_switch_utc)

### Query AWS quantum tasks

In [ ]:
tagged_tasks = braket_provider.get_tasks_by_tag("test")

len(tagged_tasks)

In [ ]:
task_arn = tagged_tasks[0]

get_quantum_task_cost(task_arn)

In [ ]:
qbraid.get_jobs(filters={"tags": {"test": "123"}})

In [ ]:
qbraid.get_jobs(filters={"vendorJobId": task_arn})

In [ ]:
qbraid_job = qbraid.job_wrapper("aws_dm_sim-markovshama-qjob-e5vxqgdjsx23tuzn1ryw")

type(qbraid_job)

In [ ]:
qbraid_job.status()

In [ ]:
qbraid_job.get_cost()

## QiskitProvider

In [ ]:
from qbraid.providers.ibm import QiskitProvider, QiskitBackend

provider = QiskitProvider()

In [ ]:
provider.get_devices(operational=True, simulator=False)

In [ ]:
provider.ibm_least_busy_qpu()

### Query IBM backends

In [ ]:
qiskit_backend = QiskitBackend(ibm_device)

qiskit_backend.provider, qiskit_backend.name

In [ ]:
qiskit_backend.status()

In [ ]:
qiskit_backend.queue_depth()

### Query IBM jobs

In [ ]:
qbraid.get_jobs(
    filters={
        "tags": {"test": "*"},
        "qbraidDeviceId": QiskitProvider.ibm_to_qbraid_id("ibmq_qasm_simulator"),
    }
)

In [ ]:
qbraid_job = qbraid.job_wrapper(
    "ibm_q_qasm_simulator-markovshama-qjob-zy12imhp7n43what3zeh"
)

type(qbraid_job)

In [ ]:
qbraid_job.status()